In [ ]:
# Import Deep Learning Library
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical

# Import Pretrained Model Library
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.callbacks import ModelCheckpoint

# Import Dataset Preprocessing Library
import scipy.io
import numpy as np
import pandas as pd

from google.colab import drive

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [ ]:
drive.mount('/content/drive', force_remount=True)

TRAIN_DIR = "/content/drive/MyDrive/Project/Dataset/train"
VAL_DIR = "/content/drive/MyDrive/Project/Dataset/val"
VAL_DIR = "/content/drive/MyDrive/Project/Dataset/test"

BATCH_SIZE = 16
IMG_SIZE = (150, 150)
NUM_CLASSES = 31

Mounted at /content/drive


In [ ]:
train_dataset = (tf.keras.utils.image_dataset_from_directory(TRAIN_DIR,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            label_mode='categorical',
                                                            seed=90))

Found 4557 files belonging to 31 classes.


In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(VAL_DIR,
                                                          shuffle=True,
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE,
                                                          label_mode='categorical',
                                                          seed=90)

Found 992 files belonging to 31 classes.


In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(VAL_DIR,
                                                          shuffle=True,
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE,
                                                          label_mode='categorical',
                                                          seed=90)

Found 992 files belonging to 31 classes.


In [ ]:
def create_augmentation_model():

    augmentation_model = tf.keras.Sequential([
        tf.keras.Input(shape=(150, 150, 3)),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomTranslation(0.1, 0.1),
        tf.keras.layers.RandomZoom(0.2)
    ])

    return augmentation_model

In [ ]:
def create_pre_trained_model():

    pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(
        include_top=False,
        input_shape=(150,150,3),
        weights='imagenet'
    )

    # Make all the layers in the pre-trained model non-trainable
    pre_trained_model.trainable = False

    return pre_trained_model

In [ ]:
def create_model():

    augmentation_layers = create_augmentation_model()
    pre_trained_model = create_pre_trained_model()

    inputs = tf.keras.Input(shape=(150,150,3))
    augmented = augmentation_layers(inputs)
    rescaled = tf.keras.layers.Rescaling(1./255)(augmented)

    x = pre_trained_model(rescaled)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.15)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    predictions = tf.keras.layers.Dense(31, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=predictions)

    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0005),
        loss="categorical_crossentropy",
        metrics=['accuracy']
    )

    return model

In [ ]:
# Save your model in a variable
model = create_model()

# Inspect parameters
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

87910968/87910968 [==============================] - 0s 0us/step
There are 41,218,879 total parameters in this model.
There are 19,416,095 trainable parameters in this model.


In [ ]:
class EarlyStoppingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy']>0.85:
            self.model.stop_training = True
            print("\nReached 99.9% accuracy so cancelling training!")

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/Project/Dataset'
checkpoint_prefix = f"{checkpoint_dir}/ckpt"

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

In [ ]:
# Muat bobot dari checkpoint terakhir
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    print(f"Loading weights from {latest_checkpoint}")
    model.load_weights(latest_checkpoint)
else:
    print("No checkpoint found. Training from scratch.")

Loading weights from /content/drive/MyDrive/Project/Dataset/ckpt


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    callbacks=[checkpoint_callback]
)

Epoch 1/5
285/285 [==============================] - ETA: 0s - loss: 0.8232 - accuracy: 0.8049
Epoch 1: val_loss did not improve from 1.04000
285/285 [==============================] - 472s 2s/step - loss: 0.8232 - accuracy: 0.8049 - val_loss: 1.0998 - val_accuracy: 0.7288
Epoch 2/5
285/285 [==============================] - ETA: 0s - loss: 0.7995 - accuracy: 0.8065
Epoch 2: val_loss did not improve from 1.04000
285/285 [==============================] - 471s 2s/step - loss: 0.7995 - accuracy: 0.8065 - val_loss: 1.4604 - val_accuracy: 0.7308
Epoch 3/5
285/285 [==============================] - ETA: 0s - loss: 0.8642 - accuracy: 0.8060
Epoch 3: val_loss improved from 1.04000 to 1.01737, saving model to /content/drive/MyDrive/Project/Dataset/ckpt
285/285 [==============================] - 470s 2s/step - loss: 0.8642 - accuracy: 0.8060 - val_loss: 1.0174 - val_accuracy: 0.7560
Epoch 4/5
285/285 [==============================] - ETA: 0s - loss: 0.7855 - accuracy: 0.8113
Epoch 4: val_loss 

In [ ]:
model.save('/content/drive/MyDrive/Project/Dataset/model2')

In [ ]:
import os
import pathlib

def list_extensions_recursive(folder_path):
    """Lists all unique file extensions in a given folder and its subfolders.

    Args:
        folder_path: The path to the folder.

    Returns:
        A dictionary mapping subfolder paths to sets of unique extensions found within them.
    """
    extensions_by_folder = {}
    for root, _, files in os.walk(folder_path):
        extensions = set()
        for file_name in files:
            file_path = os.path.join(root, file_name)
            extension = pathlib.Path(file_path).suffix.lower()
            extensions.add(extension)
        extensions_by_folder[root] = extensions  # Store extensions for this subfolder
    return extensions_by_folder

# Replace 'path/to/your/training/folder' and 'path/to/your/validation/folder' with the actual paths
training_folder = TRAIN_DIR
validation_folder = TEST_DIR

training_extensions = list_extensions_recursive(training_folder)
validation_extensions = list_extensions_recursive(validation_folder)

print("Training folder extensions:")
for folder, extensions in training_extensions.items():
    print(f"  {folder}: {extensions}")

print("\nValidation folder extensions:")
for folder, extensions in validation_extensions.items():
    print(f"  {folder}: {extensions}")

Training folder extensions:
  /content/drive/MyDrive/Project/Dataset/train: set()
  /content/drive/MyDrive/Project/Dataset/train/kunyit: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/serai: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/vanili: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/lada: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/lengkuas: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/wijen: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/kluwek: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/saffron: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/pala: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/kayu manis: {'.jpeg', '.jpg'}
  /content/drive/MyDrive/Project/Dataset/train/jinten: {'.jpeg', '.jpg'}
  /content/drive/MyDrive/Project/Dataset/train/kemukus: {'.jpeg'}
  /content/drive/MyDrive/Project/Dataset/train/jahe: {'.jpeg', '.jpg'}
  /content/drive/MyDrive/Project/Dataset/train/ke

In [ ]:
import os
import tensorflow as tf

def check_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = tf.io.read_file(file_path)
                img = tf.image.decode_image(img)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")

# Example usage
check_images(TEST_DIR)


In [ ]:
import os
from PIL import Image

def verify_and_convert_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()  # Verify that it is, in fact, an image
            img = Image.open(file_path)  # Reopen the image file
            if img.mode == 'RGBA' or img.mode not in ['RGB', 'L']:
                img = img.convert('RGB')
            if img.format != 'JPEG':
                img.save(file_path, format='JPEG')  # Save it as a proper JPEG
            #print(f"Verified and converted: {file_path}")
    except (IOError, SyntaxError) as e:
        print(f"Error processing {file_path}: {e}")

def process_images_in_directory(root_dir):
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                verify_and_convert_image(file_path)

# Example usage
root_dir = TRAIN_DIR # Replace with the path to your dataset
process_images_in_directory(root_dir)
